<a href="https://colab.research.google.com/github/snvssk/data298A/blob/development/SolarEnergy/MLModels/Solar_XGBoost_Ritanjali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df= pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/all_cities_transformed_combined.csv')
df.columns

Index(['Unnamed: 0', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature',
       'Cloud_Type', 'Dew_Point', 'GHI', 'Relative_Humidity',
       'Solar_Zenith_Angle', 'Surface_Albedo', 'Pressure',
       'Precipitable_Water', 'Wind_Direction', 'Wind_Speed', 'Location'],
      dtype='object')

In [5]:
# nsrdb_na = df.drop(columns='Unnamed: 0')
nsrdb_na = df.copy()
#null_rows = nsrdb_na[nsrdb_na.isna().any(axis=1)]
#null_rows
nsrdb_na.isnull().sum()
#nsrdb_na.columns
#nsrdb_na['GHI'].isnull().values.sum()
nsrdb = nsrdb_na.dropna()
#print(nsrdb)
nsrdb.isnull().sum()

Unnamed: 0            0
Year                  0
Month                 0
Day                   0
Hour                  0
Minute                0
Temperature           0
Cloud_Type            0
Dew_Point             0
GHI                   0
Relative_Humidity     0
Solar_Zenith_Angle    0
Surface_Albedo        0
Pressure              0
Precipitable_Water    0
Wind_Direction        0
Wind_Speed            0
Location              0
dtype: int64

In [6]:
nsrdb = nsrdb.drop(['Unnamed: 0'],axis =1)
nsrdb

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,GHI,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
0,2018.0,1.0,1.0,0.0,0.0,11.2,4.0,10.2,0.0,93.63,165.41,0.100,1024.0,1.700,294.0,0.7,117987.0
1,2018.0,1.0,1.0,0.0,30.0,11.2,4.0,10.2,0.0,93.63,165.11,0.100,1024.0,1.700,307.0,0.7,117987.0
2,2018.0,1.0,1.0,1.0,0.0,11.1,4.0,10.1,0.0,93.38,162.29,0.100,1024.0,1.700,321.0,0.7,117987.0
3,2018.0,1.0,1.0,1.0,30.0,11.0,0.0,10.1,0.0,94.00,157.91,0.100,1024.0,1.800,332.0,0.7,117987.0
4,2018.0,1.0,1.0,2.0,0.0,10.9,0.0,9.9,0.0,93.32,152.72,0.100,1024.0,1.800,344.0,0.7,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946181,2013.0,12.0,31.0,21.0,30.0,6.0,1.0,0.0,0.0,74.05,142.58,0.113,1020.0,0.950,174.5,0.6,117546.0
946182,2013.0,12.0,31.0,22.0,0.0,6.0,1.0,0.0,0.0,74.40,148.35,0.113,1020.0,0.977,152.1,0.9,117546.0
946183,2013.0,12.0,31.0,22.0,30.0,5.0,0.0,0.0,0.0,79.80,153.89,0.113,1020.0,0.972,152.1,1.1,117546.0
946184,2013.0,12.0,31.0,23.0,0.0,4.0,0.0,0.0,0.0,80.32,158.95,0.113,1020.0,0.968,166.7,1.3,117546.0


In [7]:
nsrdb = nsrdb.loc[(nsrdb['Hour'].isin([7, 10, 13, 16, 19])) & (nsrdb['Minute'] == 0)]
nsrdb

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,GHI,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
14,2018.0,1.0,1.0,7.0,0.0,9.8,7.0,8.8,0.0,93.25,94.85,0.100,1025.0,1.800,325.0,0.7,117987.0
20,2018.0,1.0,1.0,10.0,0.0,14.0,7.0,9.5,290.0,74.48,67.81,0.100,1026.0,1.800,24.0,1.5,117987.0
26,2018.0,1.0,1.0,13.0,0.0,17.6,7.0,8.6,421.0,55.38,61.32,0.100,1024.0,1.800,15.0,0.9,117987.0
32,2018.0,1.0,1.0,16.0,0.0,15.4,7.0,9.2,114.0,66.62,80.55,0.100,1023.0,1.800,14.0,0.8,117987.0
38,2018.0,1.0,1.0,19.0,0.0,12.8,7.0,8.3,0.0,74.04,112.83,0.100,1024.0,1.700,72.0,1.0,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946152,2013.0,12.0,31.0,7.0,0.0,9.0,7.0,8.0,0.0,100.00,94.75,0.111,1020.0,1.233,174.6,0.4,117546.0
946158,2013.0,12.0,31.0,10.0,0.0,13.0,3.0,7.0,281.0,74.38,67.81,0.111,1020.0,1.403,23.4,0.7,117546.0
946164,2013.0,12.0,31.0,13.0,0.0,17.0,7.0,4.0,411.0,49.14,61.44,0.111,1020.0,1.334,33.3,1.1,117546.0
946170,2013.0,12.0,31.0,16.0,0.0,7.0,4.0,1.0,84.0,73.84,80.77,0.113,1020.0,0.999,343.0,0.8,117546.0


In [8]:
target = nsrdb['GHI']#keeping only target(ghi) data here
input= nsrdb.drop(['GHI'], axis=1)
input

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
14,2018.0,1.0,1.0,7.0,0.0,9.8,7.0,8.8,93.25,94.85,0.100,1025.0,1.800,325.0,0.7,117987.0
20,2018.0,1.0,1.0,10.0,0.0,14.0,7.0,9.5,74.48,67.81,0.100,1026.0,1.800,24.0,1.5,117987.0
26,2018.0,1.0,1.0,13.0,0.0,17.6,7.0,8.6,55.38,61.32,0.100,1024.0,1.800,15.0,0.9,117987.0
32,2018.0,1.0,1.0,16.0,0.0,15.4,7.0,9.2,66.62,80.55,0.100,1023.0,1.800,14.0,0.8,117987.0
38,2018.0,1.0,1.0,19.0,0.0,12.8,7.0,8.3,74.04,112.83,0.100,1024.0,1.700,72.0,1.0,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946152,2013.0,12.0,31.0,7.0,0.0,9.0,7.0,8.0,100.00,94.75,0.111,1020.0,1.233,174.6,0.4,117546.0
946158,2013.0,12.0,31.0,10.0,0.0,13.0,3.0,7.0,74.38,67.81,0.111,1020.0,1.403,23.4,0.7,117546.0
946164,2013.0,12.0,31.0,13.0,0.0,17.0,7.0,4.0,49.14,61.44,0.111,1020.0,1.334,33.3,1.1,117546.0
946170,2013.0,12.0,31.0,16.0,0.0,7.0,4.0,1.0,73.84,80.77,0.113,1020.0,0.999,343.0,0.8,117546.0


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_normalized = sc.fit_transform(input)

In [10]:
from sklearn.model_selection import train_test_split
import numpy as np
x_train, x_test, y_train, y_test = train_test_split(input_normalized, target, test_size=0.3, random_state=42)

In [11]:
print("x_train shape",x_train.shape)
print("x_test shape",x_test.shape)
print("y_train shape",y_train.shape)
print("y_text shape",y_test.shape)

x_train shape (68985, 16)
x_test shape (29565, 16)
y_train shape (68985,)
y_text shape (29565,)


In [12]:
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=400, max_depth=3, learning_rate=0.01, colsample_bytree = 1.0, subsample = 1.0, reg_lambda = 1.5,
                              gamma = 0, tree_method='gpu_hist')

In [13]:
# fit the model
model.fit(x_train, y_train)

XGBClassifier(colsample_bytree=1.0, learning_rate=0.01, n_estimators=400,
              objective='multi:softprob', reg_lambda=1.5, subsample=1.0,
              tree_method='gpu_hist')

In [ ]:
model.save('/content/drive/MyDrive/12-8-7-3points-xgboost')

In [14]:
y_pred=model.predict(x_test)
y_pred

array([  0., 703.,   0., ..., 431., 237.,   0.])

In [15]:
from sklearn.metrics import r2_score 
R_square = r2_score(y_test, y_pred) 
print('Coefficient of Determination R2', R_square)
from sklearn import metrics

print("Model MAE", metrics.mean_absolute_error(y_test, y_pred))
print("Model MSE", metrics.mean_squared_error(y_test, y_pred))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Coefficient of Determination R2 0.8900864636014125
Model MAE 44.20284119736174
Model MSE 11440.646947404026
Model RMSE 106.96095992185198


# Cupertino Test

In [16]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/cupertino_test.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=model.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 45.664931506849314
Model MSE 11250.490136986302
Model RMSE 106.06832768072806


# FosterCity Test

In [17]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/fostercity_test.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=model.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 49.63452054794521
Model MSE 12240.21698630137
Model RMSE 110.6355141277039


# Los Gatos Test

In [18]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/losgatos_test.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=model.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 44.859452054794524
Model MSE 11135.91205479452
Model RMSE 105.5268309710593


# Saratoga

In [20]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/saratoga_2020.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
#df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=model.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 41.46739726027397
Model MSE 9686.174246575343
Model RMSE 98.41836336058095
